In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-
import time
from keras import metrics
import os
import numpy as np
import keras
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
import pandas as pd

Using TensorFlow backend.


In [22]:
# get texts data
TRAINING_PATH = './data/training/'
TESTING_PATH = './data/testing/'

categories = [dirname for dirname in os.listdir(TRAINING_PATH) if dirname[-4:] != '_cut']
# print(len(categories), str(categories))

category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}

train_pickle_df = pd.read_pickle('train.pkl')
train_texts = train_pickle_df["text"].values
train_labels = train_pickle_df["category"]
# print(len(train_texts), train_texts[0], train_labels[0])   

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values
# print(len(test_texts), test_texts[0]) 

In [99]:
# # process some data
train_labels_list = list(train_labels)
print(type(train_labels_list), len(train_labels_list), train_labels_list[0])
embedding_matrix_len = len(train_labels_list)

label_id = 0
label_list = np.zeros((embedding_matrix_len, 10))
for label_val in train_labels_list:
    index = label_val - 1
    label_list[label_id][index] = 1
    label_id += 1
print(label_list[0])
print(len(label_list)) 

<class 'list'> 9000 3
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
9000


In [69]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec1.model")
# print(type(answer))
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
# print(wvv_keys_list[:10]) #['櫻花林', '好比', '考科', '床上', '一點現', '記住', '寶寶的', '柔嫩', '不規則', '朴智妍']
# wvv_new_dict = dict(enumerate(wvv_keys_list))
# # word_index = len(wvv_keys_list)
# print(wvv_new_dict)

['櫻花林', '好比', '考科', '床上', '一點現', '記住', '寶寶的', '柔嫩', '不規則', '朴智妍']


In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
max_doc_word_length = max(len(l) for l in train_texts)
sequences1 = pad_sequences(sequences, maxlen=max_doc_word_length, padding='post')
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))
# data = pad_sequences(sequences)
# print("Shape of data tensor:" , data.shape)

Found 232236 unique tokens


In [61]:
# print(len(sequences))
print(len(sequences[0]), sequences[0])
# print(len(sequences1[0]))
# print(wvv) #'同上': <gensim.models.keyedvectors.Vocab object at 0x7fd385b03d68>, 
# print(type(wvv)) <class 'dict'>

58 [45, 40836, 2384, 40837, 5991, 49, 197, 28888, 226, 80934, 11421, 2443, 1085, 80935, 1582, 80936, 3054, 14636, 80937, 623, 6847, 13846, 1077, 80938, 80939, 2299, 80940, 80941, 80942, 2299, 129, 33600, 2868, 52, 80943, 519, 4756, 80944, 2266, 53252, 2266, 80945, 907, 5266, 80946, 33601, 80947, 49, 3840, 1714, 296, 11, 370, 836, 5890, 20, 16562, 727]


In [71]:
vocab_size = len(word_index) + 1
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 250))
for word, i in word_index.items():
    if word in wvv_keys_list:
        embedding_vector = answer[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [73]:
print(len(embedding_matrix[0]))
# input_array = np.random.randint(embedding_matrix_len, size=(50, 3971))
# print(input_array.shape)

250


In [100]:
# input_dim = 400
# output_dim = 100
embedding_layer = Embedding(vocab_size, 250, 
                            weights=[embedding_matrix], 
                            input_length= max_doc_word_length, 
                            trainable=False)
model = Sequential()
model.add(embedding_layer)

model.add(SimpleRNN(input_dim = 250, output_dim = 50, unroll=True))
nb_classes = 10
model.add(Dense( nb_classes, input_dim = 3971))
model.add(Activation('softmax'))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=[metrics.mae, metrics.categorical_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3971, 250)         58059250  
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 50)                15050     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                510       
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 58,074,810
Trainable params: 15,560
Non-trainable params: 58,059,250
_________________________________________________________________


In [ ]:
history = model.fit(x = sequences1, y = label_list, 
                    validation_split=0.1, 
                    callbacks=[early_stopping], 
                    epochs = 1, verbose = 0)

Train on 8100 samples, validate on 900 samples
Epoch 1/1
3200/8100 [==========>...................] - ETA: 5:09 - loss: 2.2602 - mean_absolute_error: 0.1789 - categorical_accuracy: 0.1072

In [ ]:
test_feature=  embedding_matrix[5010:5011]
predict_res = model.predict(test_feature, batch_size= 32, verbose=0)
print(len(predict_res), predict_res)

final_res = []
now_item = predict_res[0]
max_val = max(now_item)
final_res.append(np.argmax(now_item))
print(final_res)

In [ ]:
# del model
# model = load_model('my_model.h5')
len(embedding_matrix)

In [ ]:
# final_res = []
# print(len(test_embedding_matrix))
# for predicc in test_embedding_matrix:
#     test_feature= np.asarray(predicc)
#     predict_res = model.predict(test_feature, batch_size= 32, verbose=0)
# #     print(len(predict_res), predict_res[0])
    
#     now_item = predict_res[0]
#     max_val = max(now_item)
#     final_res.append(np.argmax(now_item))
# print(final_res)

In [ ]:
# # result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
# print(len(final_res))
# result_txt = "result001" + ".txt"
# ids = 0
# with open(result_txt, 'a') as out:
#     out.write("id,category" + '\n')
#     for value in final_res:
#         out.write(str(ids) + "," + str(value) + '\n')
#         ids += 1

In [ ]:
# start_time = time.time()
# start = 0

# for embed_item_index in range(0, 487):
# #     train_result= np.asarray([embedding_matrix])
# #     train_label =  np.asarray([label_list[label_id]])
    
#     end = start + 100
#     train_result= articles_matrix[start:end]
#     train_label = label_list[start:end]
#     history = model.fit(x = train_result, y = train_label, validation_split=0.1, callbacks=[early_stopping], epochs = 5, verbose = 0)
#     model.save('my_model.h5')
#     scores = model.evaluate(x = train_result, y = train_label, batch_size=32, verbose = 0)
#     if  embed_item_index % 10 == 0 :  
#         end_time = time.time()
#         excution_time = end_time- start_time
#         excution_time_min = round(excution_time / 60, 2)
#         print(embed_item_index, " , loss : ", round(scores[0], 6), "   acc : " , scores[1],
#               "  minutes: " , excution_time_min)
# #     loss_func_name = 'categorical_crossentropy'
# # #     for value in history.history[loss_func_name]:
# #     print(history)
        
#     if embed_item_index < 40 :  
#         pass
        
#     else:
#         break
#     start += 100